In [14]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

train = pd.read_csv('./data/finger_atom_pair_train.csv')
train.drop(columns=['id','SMILES'],inplace=True)
train = train.rename(columns = {'Num_H_Acceptors': 'Acceptors','Num_H_Donors': 'Donors','Num_RotatableBonds':'RotatableBonds'})


In [15]:
import optuna
import optuna.logging
from tqdm import tqdm
from sklearn.model_selection import train_test_split
def hype_tune(col):
    x_train, x_valid, y_train, y_valid = train_test_split(train.drop(columns=['HLM',"MLM"]),train[f'{col}'], test_size=0.33, random_state=42)

    dtrain = xgb.DMatrix(data=x_train, label=y_train, enable_categorical=True)
    dvalid = xgb.DMatrix(data=x_valid, label=y_valid, enable_categorical=True)

    def objective(trial):
        param = {
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0),
            'gamma': trial.suggest_float('gamma', 1e-3, 10),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree',[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
            'max_depth': trial.suggest_categorical('max_depth', [3, 4, 5, 6]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
            'eta' : 0.1,
            'tree_method' : "hist",
            'gpu_id': 0
        }

        model = xgb.train(params=param, dtrain=dtrain, num_boost_round=1000,
                        evals=[(dvalid, 'valid')], early_stopping_rounds=50, verbose_eval=False)

        preds = model.predict(dvalid)
        rmse = mean_squared_error(y_valid, preds, squared=False)
        return rmse
        

        

    study = optuna.create_study(direction='minimize', study_name=None)
    with tqdm(total=500) as pbar:  
        def callback(study, trial):
            pbar.update(1)  

        study.optimize(objective, n_trials=500, callbacks=[callback])
        
    df = study.trials_dataframe().sort_values(by=['value'], ascending=[True]).reset_index(drop=True)
    best = study.best_trial
    print("best:",best)
    #df.to_csv('parameters.csv', index=False)
    df.head(5)
    return best

MLM

In [16]:
mlm_best = hype_tune("MLM")

[I 2023-09-15 13:13:53,945] A new study created in memory with name: no-name-348b5eae-b260-4a5c-84c7-8b028e4b2195
  0%|          | 0/500 [00:00<?, ?it/s]

[13:13:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[I 2023-09-15 13:13:59,892] Trial 0 finished with value: 32.560462075564644 and parameters: {'reg_lambda': 5.78491611664917, 'gamma': 2.7203442118436643, 'reg_alpha': 7.799670827955122, 'colsample_bytree': 0.7, 'subsample': 0.8, 'max_depth': 4, 'min_child_weight': 172}. Best is trial 0 with value: 32.560462075564644.
  0%|          | 1/500 [00:05<49:21,  5.94s/it]

[13:13:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[I 2023-09-15 13:14:06,507] Trial 1 finished with value: 32.769796753151084 and parameters: {'reg_lambda': 4.192366296904137, 'gamma': 4.871595952762468, 'reg_alpha': 7.105891003120269, 'colsample_bytree': 1.0, 'subsample': 0.6, 'max_depth': 4, 'min_child_weight': 293}. Best is trial 0 with value: 32.560462075564644.
  0%|          | 2/500 [00:12<52:35,  6.34s/it]

...

[13:46:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[I 2023-09-15 13:46:16,241] Trial 317 finished with value: 32.36641698313764 and parameters: {'reg_lambda': 0.35989357658541826, 'gamma': 6.781227585015928, 'reg_alpha': 3.316561119311108, 'colsample_bytree': 0.7, 'subsample': 1.0, 'max_depth': 6, 'min_child_weight': 159}. Best is trial 206 with value: 31.98428935348078.
 64%|██████▎   | 318/500 [32:22<19:08,  6.31s/it]

[13:46:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[W 2023-09-15 13:46:18,792] Trial 318 failed with parameters: {'reg_lambda': 2.8189975744854894, 'gamma': 8.158329649544386, 'reg_alpha': 3.8597843588199234, 'colsample_bytree': 0.7, 'subsample': 1.0, 'max_depth': 5, 'min_child_weight': 150} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Admin\.conda\envs\poum\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_8536\1350875920.py", line 25, in objective
    model = xgb.train(params=param, dtrain=dtrain, num_boost_round=1000,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\.conda\envs\poum\Lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\Admin\.conda\envs\poum\Lib\site-packages\xgboost\training.py", line 185, in train
   

KeyboardInterrupt: 

HLM

In [ ]:
hlm_best = hype_tune("HLM")

[I 2023-09-15 12:08:35,862] A new study created in memory with name: no-name-52ae5fa2-ba6b-47b0-9eee-c5214667b6ee
  0%|          | 0/500 [00:00<?, ?it/s]

[12:08:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[I 2023-09-15 12:08:36,327] Trial 0 finished with value: 32.78773522960385 and parameters: {'reg_lambda': 0.5338300290164114, 'gamma': 9.19237382070738, 'reg_alpha': 6.358304634831731, 'colsample_bytree': 0.9, 'subsample': 1.0, 'max_depth': 6, 'min_child_weight': 220}. Best is trial 0 with value: 32.78773522960385.
  0%|          | 1/500 [00:00<03:48,  2.18it/s]

[12:08:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[I 2023-09-15 12:08:36,758] Trial 1 finished with value: 32.69926651838628 and parameters: {'reg_lambda': 4.141147908330912, 'gamma': 2.5360874668894295, 'reg_alpha': 6.997298887444267, 'colsample_bytree': 0.9, 'subsample': 0.8, 'max_depth': 6, 'min_child_weight': 23}. Best is trial 1 with value: 32.69926651838628.
  0%|          | 2/500 [00:00<03:40,  2.26it/s]

...

[12:13:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[I 2023-09-15 12:13:21,248] Trial 498 finished with value: 32.348682530148054 and parameters: {'reg_lambda': 8.534200738051664, 'gamma': 9.624682066278481, 'reg_alpha': 9.325955336846016, 'colsample_bytree': 1.0, 'subsample': 0.8, 'max_depth': 3, 'min_child_weight': 35}. Best is trial 122 with value: 32.19141259019123.
100%|█████████▉| 499/500 [04:45<00:00,  1.69it/s]

[12:13:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:339: No visible GPU is found, setting `gpu_id` to -1


[I 2023-09-15 12:13:21,976] Trial 499 finished with value: 32.459230360040124 and parameters: {'reg_lambda': 7.976959249587276, 'gamma': 8.59420802758548, 'reg_alpha': 8.091793208116373, 'colsample_bytree': 1.0, 'subsample': 0.8, 'max_depth': 3, 'min_child_weight': 20}. Best is trial 122 with value: 32.19141259019123.
100%|██████████| 500/500 [04:46<00:00,  1.75it/s]

best: FrozenTrial(number=122, state=1, values=[32.19141259019123], datetime_start=datetime.datetime(2023, 9, 15, 12, 9, 38, 441214), datetime_complete=datetime.datetime(2023, 9, 15, 12, 9, 38, 908751), params={'reg_lambda': 7.907577397615725, 'gamma': 9.61658605798799, 'reg_alpha': 9.441643680941189, 'colsample_bytree': 1.0, 'subsample': 0.8, 'max_depth': 3, 'min_child_weight': 36}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reg_lambda': FloatDistribution(high=10.0, log=False, low=0.001, step=None), 'gamma': FloatDistribution(high=10.0, log=False, low=0.001, step=None), 'reg_alpha': FloatDistribution(high=10.0, log=False, low=0.001, step=None), 'colsample_bytree': CategoricalDistribution(choices=(0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'subsample': CategoricalDistribution(choices=(0.6, 0.7, 0.8, 1.0)), 'max_depth': CategoricalDistribution(choices=(3, 4, 5, 6)), 'min_child_weight': IntDistribution(high=300, log=False, low=1, step=1)}, trial_id=122, value=None)


In [ ]:
print("mlm",mlm_best.values)
print(mlm_best.params)
print("hlm",hlm_best.values)
print(hlm_best.params)

mlm [32.412156466039214]
{'reg_lambda': 3.970598535915323, 'gamma': 9.254009054984513, 'reg_alpha': 8.180688938901085, 'colsample_bytree': 0.7, 'subsample': 0.7, 'max_depth': 3, 'min_child_weight': 158}
hlm [32.19141259019123]
{'reg_lambda': 7.907577397615725, 'gamma': 9.61658605798799, 'reg_alpha': 9.441643680941189, 'colsample_bytree': 1.0, 'subsample': 0.8, 'max_depth': 3, 'min_child_weight': 36}
